In [1]:
library(loo)
library(rstan)

This is loo version 2.0.0.
**NOTE: As of version 2.0.0 loo defaults to 1 core but we recommend using as many as possible. Use the 'cores' argument or set options(mc.cores = NUM_CORES) for an entire session. Visit mc-stan.org/loo/news for details on other changes.


ERROR: Error in library(rstan): there is no package called ‘rstan’


In [2]:
install.packages("rstan", dependencies=TRUE)

also installing the dependencies ‘miniUI’, ‘htmlwidgets’, ‘crosstalk’, ‘PKI’, ‘RCurl’, ‘RJSONIO’, ‘igraph’, ‘colourpicker’, ‘DT’, ‘dygraphs’, ‘gtools’, ‘rsconnect’, ‘shinyjs’, ‘shinythemes’, ‘threejs’, ‘shinystan’

Warning message in install.packages("rstan", dependencies = TRUE):
“installation of package ‘miniUI’ had non-zero exit status”Warning message in install.packages("rstan", dependencies = TRUE):
“installation of package ‘crosstalk’ had non-zero exit status”Warning message in install.packages("rstan", dependencies = TRUE):
“installation of package ‘PKI’ had non-zero exit status”Warning message in install.packages("rstan", dependencies = TRUE):
“installation of package ‘RCurl’ had non-zero exit status”Warning message in install.packages("rstan", dependencies = TRUE):
“installation of package ‘RJSONIO’ had non-zero exit status”Warning message in install.packages("rstan", dependencies = TRUE):
“installation of package ‘igraph’ had non-zero exit status”Warning message in install.pa